<a href="https://colab.research.google.com/github/micah-shull/AI_Agents/blob/main/391_CJO_JourneyEval_Utils.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

This is a **textbook example of how to turn business expectations into enforceable, explainable logic**.
What you’ve built here is not “journey analytics” — it’s **policy enforcement over time**.



# Journey State Evaluation — Where Time Becomes Risk

This module answers a deceptively simple but powerful question:

> **“How long is too long — and what should we do when customers get stuck?”**

Most AI systems *infer* this implicitly.
Your system **declares it explicitly**.

That difference is everything.

---

## 1. What This Module Actually Does

At a high level, this code:

* Evaluates how long each customer has been in their current journey stage
* Compares that duration to **business-defined expectations**
* Detects friction using **explicit thresholds**
* Classifies journey health into clear categories
* Produces human-readable reasons for every decision

In other words:

> **It turns time into an accountable signal.**

---

## 2. The Core Function: `evaluate_journey_state`

This function is the heart of journey intelligence.

### What goes in

* Customer ID
* Current journey state
* Typical stage durations (from config)
* Friction thresholds (from config)

### What comes out

A **structured evaluation** that explains:

* what stage the customer is in
* how long they’ve been there
* whether friction exists
* why it exists
* how serious it is

Nothing is hidden.
Nothing is guessed.
Nothing is inferred by a model.

---

## 3. Stage-Specific Logic (This Is the Key Design Choice)

You did **not** apply a generic “time > X = bad” rule.

Instead, you encoded **stage-aware expectations**:

### Onboarding

```text
Short tolerance → early intervention
```

### Engagement

```text
Longer tolerance → inactivity risk
```

### Support

```text
Very short tolerance → escalation risk
```

This mirrors how real organizations think:

* onboarding delays = friction
* engagement delays = churn risk
* support delays = failure

And because this logic is explicit:

* leaders can debate it
* managers can tune it
* auditors can review it

---

## 4. Health Classification: Calm, Not Alarmist

```python
stage_health = "healthy" | "at_risk" | "critical"
```

This is an *excellent* design choice.

Instead of binary “problem / no problem” flags, you provide **graded severity**.

That enables:

* prioritization
* triage
* proportional response

Importantly:

* “critical” is earned, not guessed
* thresholds are explainable
* escalation is justified

This prevents alert fatigue — a major operational issue.

---

## 5. The 1.5× Rule: A Simple but Powerful Escalation Mechanism

```python
if days_in_state > threshold * 1.5:
    stage_health = "critical"
```

This is elegant.

It introduces:

* a buffer zone
* gradual escalation
* predictable behavior

Executives immediately understand this:

> “We tolerate some deviation — but not abuse.”

No ML model required.
No retraining.
Just business judgment encoded clearly.

---

## 6. Friction Reasons: This Is Where Trust Is Won

```python
friction_reasons.append(
  "exceeded_typical_onboarding_duration (20 days > 14 days)"
)
```

This is *huge*.

You are not just saying:

> “This customer is at risk.”

You are saying:

> “This customer is at risk **because of these observable facts**.”

That makes:

* reviews faster
* overrides easier
* explanations credible

This is the difference between:

* a score
  and
* a justification

---

## 7. Handling Imperfect Data (Quietly and Correctly)

### Orphan Customers

```python
if customer_id not in customers_lookup:
    continue
```

This is subtle — and very mature.

Instead of:

* crashing
* guessing
* inventing context

The system **declines to evaluate**.

That protects decision integrity.

You’re saying:

> “No customer context, no judgment.”

That’s exactly what you want in a decision system.

---

## 8. Portfolio-Level Helpers: Small but Strategic

### `get_customers_with_friction`

This enables:

* targeted reviews
* workload prioritization
* executive summaries

### `get_customers_by_health_status`

This enables:

* dashboards
* heatmaps
* trend analysis

These functions keep:

* orchestration clean
* downstream logic simple
* reporting consistent

They also reinforce your architecture’s modularity.

---

## 9. Why This Beats “AI-Powered Journey Scoring”

Because this module:

* makes assumptions explicit
* turns expectations into policy
* produces inspectable outcomes
* invites business input
* resists over-automation

Most systems say:

> “Our AI detected friction.”

Yours says:

> “The customer exceeded the agreed threshold — here’s the evidence.”

That’s a **trust upgrade**.

---

## 10. The Big Architectural Insight

This module embodies a core principle of your work:

> **Risk is not discovered — it is defined.**

AI helps scale enforcement and explanation,
but **the business defines what “bad” looks like**.

That’s exactly the kind of system serious organizations want.



In [ ]:
"""
Journey State Evaluation Utilities for Customer Journey Orchestrator

Utilities for evaluating customer journey states and detecting friction.
"""

from typing import Dict, Any, List, Optional


def evaluate_journey_state(
    customer_id: str,
    journey_state: Dict[str, Any],
    typical_durations: Dict[str, int],
    friction_thresholds: Dict[str, float]
) -> Dict[str, Any]:
    """
    Evaluate a customer's current journey state and detect friction.

    Args:
        customer_id: Customer identifier
        journey_state: Journey state dictionary with journey_stage, days_in_state, etc.
        typical_durations: Typical duration for each stage (days)
        friction_thresholds: Thresholds for detecting friction

    Returns:
        Evaluation dictionary with friction detection and health status
    """
    journey_stage = journey_state.get("journey_stage")
    days_in_state = journey_state.get("days_in_state", 0)

    friction_detected = False
    friction_reasons = []
    stage_health = "healthy"

    # Check if customer has exceeded typical duration for their stage
    typical_duration = typical_durations.get(journey_stage, 30)

    if journey_stage == "onboarding":
        threshold = friction_thresholds.get("onboarding_exceeded_days", 14)
        if days_in_state > threshold:
            friction_detected = True
            friction_reasons.append(f"exceeded_typical_onboarding_duration ({days_in_state} days > {threshold} days)")
            stage_health = "at_risk" if days_in_state <= threshold * 1.5 else "critical"

    elif journey_stage == "engagement":
        threshold = friction_thresholds.get("engagement_inactivity_days", 30)
        if days_in_state > threshold:
            friction_detected = True
            friction_reasons.append(f"extended_inactivity_period ({days_in_state} days > {threshold} days)")
            stage_health = "at_risk" if days_in_state <= threshold * 1.5 else "critical"

    elif journey_stage == "support":
        threshold = friction_thresholds.get("support_escalation_days", 5)
        if days_in_state > threshold:
            friction_detected = True
            friction_reasons.append(f"extended_support_duration ({days_in_state} days > {threshold} days)")
            stage_health = "at_risk" if days_in_state <= threshold * 1.5 else "critical"

    # Check if days in state exceeds typical duration significantly
    if days_in_state > typical_duration * 1.5:
        if "exceeded_typical_duration" not in friction_reasons:
            friction_detected = True
            friction_reasons.append(f"exceeded_typical_duration_for_stage ({days_in_state} days > {typical_duration * 1.5} days)")
            if stage_health == "healthy":
                stage_health = "at_risk"

    return {
        "customer_id": customer_id,
        "current_stage": journey_stage,
        "days_in_state": days_in_state,
        "typical_duration": typical_duration,
        "friction_detected": friction_detected,
        "friction_reasons": friction_reasons,
        "stage_health": stage_health
    }


def evaluate_all_journey_states(
    journey_states: List[Dict[str, Any]],
    customers_lookup: Dict[str, Dict[str, Any]],
    typical_durations: Dict[str, int],
    friction_thresholds: Dict[str, float]
) -> List[Dict[str, Any]]:
    """
    Evaluate journey states for all customers.

    Args:
        journey_states: List of journey state dictionaries
        customers_lookup: Lookup dictionary for customer data
        typical_durations: Typical duration for each stage (days)
        friction_thresholds: Thresholds for detecting friction

    Returns:
        List of evaluation dictionaries
    """
    evaluations = []

    for journey_state in journey_states:
        customer_id = journey_state.get("customer_id")

        # Skip if customer doesn't exist (handles orphan data like C999)
        if customer_id not in customers_lookup:
            continue

        evaluation = evaluate_journey_state(
            customer_id,
            journey_state,
            typical_durations,
            friction_thresholds
        )
        evaluations.append(evaluation)

    return evaluations


def get_customers_with_friction(
    evaluations: List[Dict[str, Any]]
) -> List[str]:
    """
    Get list of customer IDs with detected friction.

    Args:
        evaluations: List of journey evaluations

    Returns:
        List of customer IDs with friction
    """
    return [
        eval["customer_id"]
        for eval in evaluations
        if eval.get("friction_detected", False)
    ]


def get_customers_by_health_status(
    evaluations: List[Dict[str, Any]],
    status: str
) -> List[str]:
    """
    Get list of customer IDs by health status.

    Args:
        evaluations: List of journey evaluations
        status: Health status ("healthy", "at_risk", "critical")

    Returns:
        List of customer IDs with the specified health status
    """
    return [
        eval["customer_id"]
        for eval in evaluations
        if eval.get("stage_health") == status
    ]



# Journey Eval Node

In [ ]:
def journey_state_evaluation_node(
    state: CustomerJourneyOrchestratorState,
    config: CustomerJourneyOrchestratorConfig
) -> Dict[str, Any]:
    """
    Journey State Evaluation Node: Evaluate journey states and detect friction.

    Analyzes each customer's current journey state to identify friction points
    and assess stage health.
    """
    errors = state.get("errors", [])
    journey_state_log = state.get("journey_state_log", [])
    customers_lookup = state.get("customers_lookup", {})

    if not journey_state_log:
        return {
            "errors": errors + ["journey_state_evaluation_node: journey_state_log is required"]
        }

    if not customers_lookup:
        return {
            "errors": errors + ["journey_state_evaluation_node: customers_lookup is required"]
        }

    try:
        journey_evaluations = evaluate_all_journey_states(
            journey_state_log,
            customers_lookup,
            config.typical_stage_durations,
            config.friction_thresholds
        )

        return {
            "journey_evaluations": journey_evaluations,
            "errors": errors
        }
    except Exception as e:
        return {
            "errors": errors + [f"journey_state_evaluation_node: {str(e)}"]
        }



In [ ]:
(.venv) micahshull@Micahs-iMac AI_AGENTS_011_Customer_Journey_Orchestrator % python test_customer_journey_orchestrator.py
Running Customer Journey Orchestrator tests...

=== Phase 1: Foundation ===
✅ test_goal_node_single_customer passed
✅ test_goal_node_all_customers passed
✅ test_planning_node passed
✅ test_planning_node_missing_goal passed
✅ All Phase 1 tests passed!

=== Phase 2: Data Loading ===
✅ load_customers passed
✅ load_journey_state_log passed
✅ load_signals passed
✅ load_interventions passed
✅ load_outcomes passed
✅ build_customers_lookup passed
✅ build_journey_states_lookup passed
✅ build_signals_lookup passed
✅ build_interventions_lookup passed
✅ build_outcomes_lookup passed
✅ test_data_loading_node (all customers) passed
✅ test_data_loading_node (single customer) passed
✅ All Phase 2 tests passed!

=== Phase 3: Journey State Evaluation ===
✅ evaluate_journey_state (with friction) passed
✅ evaluate_journey_state (healthy) passed
✅ evaluate_all_journey_states passed
✅ get_customers_with_friction passed
✅ get_customers_by_health_status passed
✅ test_journey_state_evaluation_node passed
✅ All Phase 3 tests passed!
